In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import patsy
from patsy import dmatrices, dmatrix
%matplotlib inline

In [2]:
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('Titanic Dataset.csv')

In [6]:
print(" Data Summary\n \n ", df.isnull().sum())

 Data Summary
 
  PassengerId      0
Survived         0
Pclass           0
Name             0
Gender           0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [7]:
df.tail(10)

,PassengerId,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
881,882,0,3,"Markun, Mr. Johann",male,33.0,0,0,349257,7.8958,NaN,S
882,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,NaN,S
883,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.5000,NaN,S
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.7500,NaN,Q


In [8]:
df = df.fillna(method='bfill')
df = df.fillna(method='ffill')

/var/folders/8y/m7b7vwqn1gq3s5gg_9vtr5b80000gn/T/ipykernel_11110/2722482854.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill')
/var/folders/8y/m7b7vwqn1gq3s5gg_9vtr5b80000gn/T/ipykernel_11110/2722482854.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


In [9]:
print("Data Summary\n \n ", df.isnull().sum())

Data Summary
 
  PassengerId    0
Survived       0
Pclass         0
Name           0
Gender         0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


In [10]:
df.head(20)

,PassengerId,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,C85,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,C123,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,E46,S
5,6,0,3,"Moran, Mr. James",male,54.0,0,0,330877,8.4583,E46,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,G6,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,G6,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,G6,C


In [11]:
y,X = dmatrices('Survived ~ Pclass', data = df)

In [12]:
type(y)

patsy.design_info.DesignMatrix

In [13]:
type(X)


patsy.design_info.DesignMatrix

In [14]:
X.shape

(891, 2)

In [15]:
(891, 2)


(891, 2)

In [16]:
X[:5]

array([[1., 3.],
       [1., 1.],
       [1., 3.],
       [1., 1.],
       [1., 3.]])

In [17]:
y.shape

(891, 1)

In [18]:
y[:5]

array([[0.],
       [1.],
       [1.],
       [1.],
       [0.]])

In [19]:
y[:5]

array([[0.],
       [1.],
       [1.],
       [1.],
       [0.]])

In [20]:
y,X = dmatrices('Survived ~ Pclass+Age', data = df)

In [21]:
y,X = dmatrices('Survived ~ Pclass+Age -1', data = df)

In [22]:
X = dmatrix('Pclass + Age', data= df)

In [23]:
X = dmatrix('standardize(Pclass)+Age', data=df)

In [24]:
X[:5]

array([[ 1.        ,  0.82737724, 22.        ],
       [ 1.        , -1.56610693, 38.        ],
       [ 1.        ,  0.82737724, 26.        ],
       [ 1.        , -1.56610693, 35.        ],
       [ 1.        ,  0.82737724, 35.        ]])

In [25]:
df.Gender.dtype

dtype('O')

In [26]:
df.Gender.unique()

array(['male', 'female'], dtype=object)

In [28]:
df.Cabin.unique()

array(['C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6', 'C23 C25 C27',
       'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33', 'F G73', 'E31',
       'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101', 'F E69', 'D47',
       'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4', 'A32', 'B4',
       'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35', 'C87', 'B77',
       'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19', 'B49', 'D',
       'C22 C26', 'C106', 'C65', 'E36', 'C54', 'B57 B59 B63 B66', 'C7',
       'E34', 'C32', 'B18', 'C124', 'C91', 'E40', 'T', 'C128', 'D37',
       'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44', 'A34', 'C104',
       'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14', 'B37', 'C30',
       'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38', 'B39', 'B22',
       'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68', 'B41', 'A20',
       'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48', 'E58', 'C126',
       'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63', 'C62 C64',
       'E24

In [29]:
df.Embarked.unique()

array(['S', 'C', 'Q'], dtype=object)

In [30]:
X = dmatrix('C(Embarked)+Pclass-1', data = df)

In [31]:
X_train, X_test,y_train, y_test = train_test_split(X,y, test_size = 0.25)

In [32]:
X_train

array([[0., 0., 1., 3.],
       [0., 1., 0., 3.],
       [0., 0., 1., 1.],
       ...,
       [0., 0., 1., 3.],
       [0., 0., 1., 2.],
       [1., 0., 0., 1.]])

In [33]:
X_test

array([[0., 1., 0., 3.],
       [0., 0., 1., 3.],
       [1., 0., 0., 3.],
       [1., 0., 0., 1.],
       [0., 0., 1., 2.],
       [0., 0., 1., 3.],
       [0., 0., 1., 3.],
       [1., 0., 0., 1.],
       [0., 1., 0., 3.],
       [0., 0., 1., 1.],
       [0., 1., 0., 3.],
       [0., 0., 1., 3.],
       [0., 0., 1., 3.],
       [1., 0., 0., 1.],
       [0., 0., 1., 2.],
       [1., 0., 0., 1.],
       [1., 0., 0., 1.],
       [1., 0., 0., 1.],
       [0., 0., 1., 3.],
       [0., 0., 1., 3.],
       [0., 0., 1., 3.],
       [0., 0., 1., 1.],
       [0., 0., 1., 1.],
       [0., 0., 1., 3.],
       [0., 0., 1., 3.],
       [0., 0., 1., 3.],
       [0., 0., 1., 1.],
       [0., 0., 1., 3.],
       [0., 0., 1., 2.],
       [0., 0., 1., 3.],
       [1., 0., 0., 2.],
       [0., 0., 1., 3.],
       [0., 0., 1., 3.],
       [0., 0., 1., 2.],
       [1., 0., 0., 3.],
       [0., 0., 1., 1.],
       [0., 0., 1., 2.],
       [1., 0., 0., 1.],
       [0., 0., 1., 1.],
       [1., 0., 0., 3.],


In [34]:
y_train

array([[0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],

In [35]:
y_test

array([[1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],

In [36]:
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score

In [37]:
model = LinearRegression(normalize=True)

TypeError: LinearRegression.__init__() got an unexpected keyword argument 'normalize'